<div style="width:100%; background-color: #D9EDF7; border: 1px solid #CFCFCF; text-align: left; padding: 10px;">
      <b>Weather Data: Renewables.ninja processing notebook</b>
      <ul>
        <li><a href="main.ipynb">Main notebook</a></li>
        <li><a href="download.ipynb">Download weather data from Renewables.ninja</a></li>
        <li>Process weather data from Renewables.ninja</li>
        <li><a href="download_merra2.ipynb">Example script to download arbitrary MERRA-2 data</a></li>
      </ul>
      <br>This Notebook is part of the <a href="http://data.open-power-system-data.org/weather_data">Weather Data Package</a> of <a href="http://open-power-system-data.org">Open Power System Data</a>.
</div>

In [1]:
%load_ext autoreload
%autoreload 2

import glob
import os
import sqlite3
import hashlib
import shutil

import pandas as pd
import geopandas as gp
import gsee
import tqdm

import generate_metadata

In [2]:
version = '2018-08-30'
changes = 'Initial release'

In [3]:
dir_shapefiles = './downloads/shapefiles'
dir_countries = './downloads/countries'
dir_nuts = './downloads/nuts'

# Read and process data

In [4]:
dataframes = {}
parse_kwargs = dict(skiprows=2, index_col=0, parse_dates=True)

# Files of form `ninja_weather_country_AT_merra-2_population_weighted.csv`
for f in glob.glob(os.path.join(dir_countries, '*.csv')):

    country_code = f.split('_')[3]

    df = pd.read_csv(f, **parse_kwargs)
    
    dataframes[country_code] = df
    
# Files of form `ninja_weather_irradiance_surface_country_DE_merra-2_nuts-2_population_weighted.csv`
for f in glob.glob(os.path.join(dir_nuts, '*.csv')):

    country_code = f.split('country_')[1][0:2]
    variable = f.split('weather_')[1].split('_country')[0]

    df = pd.read_csv(f, **parse_kwargs)
    df = df.rename(columns={country_code + '_TOTAL': country_code})

    for c in df.columns:
        if c not in dataframes:
            dataframes[c] = pd.DataFrame({variable: df[c]})
        else:
            dataframes[c].loc[:, variable] = df[c]


In [5]:
# Estimate direct and diffuse radiation using the BRL model as implented in GSEE.
# https://github.com/renewables-ninja/gsee

# Warning: this code takes a while to execute (easily ~60 seconds per location
# even on a reasonably fast machine).

# Subset `dataframes` to speed up for testing, e.g.:
# `dataframes = {k: v for k, v in dataframes.items() if k in ['DEA2', 'DEA3', 'BE']}`

nuts_centroids = gp.GeoDataFrame.from_file(os.path.join(dir_shapefiles, 'NUTS_LB_2016_4326.shp'))
nuts_centroids.set_index('NUTS_ID', inplace=True)

for k in tqdm.tqdm(dataframes.keys()):
    df = dataframes[k]
    diffuse_fraction = gsee.brl_model.run(
        hourly_clearness=df['irradiance_surface'] / df['irradiance_toa'],
        # lat, lon of centroid
        coords=list(nuts_centroids.loc[country_code, 'geometry'].coords)[0][::-1]
    )

    df['radiation_direct_horizontal'] = ((1 - diffuse_fraction) * df['irradiance_surface']).fillna(0)
    df['radiation_diffuse_horizontal'] = (diffuse_fraction * df['irradiance_surface']).fillna(0)
    dataframes[k] = df


100%|██████████| 48/48 [52:06<00:00, 65.14s/it]


In [6]:
variables = ['temperature', 'radiation_direct_horizontal', 'radiation_diffuse_horizontal']

for k in dataframes.keys():
    dataframes[k] = dataframes[k].loc[:, [v for v in variables if v in dataframes[k].columns]]

In [7]:
complete_data = pd.concat(dataframes, axis=1, join='inner')

In [17]:
df = complete_data

In [18]:
df.columns = pd.MultiIndex.from_tuples(
    [(i[0], i[1]) for i in df.columns],
    names=['geography', 'variable']
)

# Write data to disk

## Reshape data

Data are provided in three different "shapes": 
- SingleIndex (easy to read for humans, compatible with datapackage standard, small file size)
  - File format: CSV, SQLite
- MultiIndex (easy to read into GAMS, not compatible with datapackage standard, small file size)
  - File format: CSV, Excel
- Stacked (compatible with data package standard, large file size, many rows, too many for Excel) 
  - File format: CSV

In [19]:
df_multiindex = df

df_singleindex = df.copy()

df_singleindex.columns = [
    '_'.join([level for level in list(col)])
    for col in df.columns.values
]

df_stacked = df.copy()
df_stacked = df_stacked.transpose().stack(dropna=True).to_frame(name='data')

## Write to SQLite database

In [20]:
# SQLite is required for the filtering function on the OPSD website

df = df_singleindex.copy()
df.index = df.index.strftime('%Y-%m-%dT%H:%M:%SZ')
filepath = os.path.join(version, 'weather_data.sqlite')
df.to_sql(
    'weather_data_singleindex',
    sqlite3.connect(filepath),
    if_exists='replace',
    index_label='time'
)

## Write to CSV

In [21]:
def save_df(df, stacking_key):
    
    filepath = os.path.join(version, 'weather_data_' + stacking_key + '.csv')

    df.to_csv(filepath, float_format='%.4f',
              date_format='%Y-%m-%dT%H:%M:%SZ')
    
save_df(df_multiindex, 'multiindex')
save_df(df_singleindex, 'singleindex')
# save_df(df_stacked, 'stacked')  # Disabled - file too large

## Write metadata

In [22]:
# See generate_metadata.py for details
generate_metadata.generate_json(df_multiindex, version, changes)

## Write checksums.txt

We publish SHA checksums for the output files on GitHub to allow verifying their integrity on the OPSD server.

In [23]:
def get_sha_hash(path, blocksize=65536):
    sha_hasher = hashlib.sha256()
    with open(path, 'rb') as f:
        buffer = f.read(blocksize)
        while len(buffer) > 0:
            sha_hasher.update(buffer)
            buffer = f.read(blocksize)
        return sha_hasher.hexdigest()


checksum_file_path = os.path.join(version, 'checksums.txt')
files = glob.glob(os.path.join(version, 'weather_data*'))

# Create checksums.txt in the version directory
with open(checksum_file_path, 'w') as f:
    for this_file in files:
        file_hash = get_sha_hash(this_file)
        f.write('{},{}\n'.format(os.path.basename(this_file), file_hash))

# Copy the file to root directory from where it will be pushed to GitHub
shutil.copyfile(checksum_file_path, 'checksums.txt')

'checksums.txt'